In [186]:
class MeasureUnit():
    
    def __init__(self,value,unit):
        self.value = value
        
        if type(unit) == list:
            self.unit = list(unit)
        
        elif type(unit) == str:
            C = unit.replace("**","^").split("*")
            D = []
            for c in C:
                if c.count("/") == 0:
                    pass
                elif c.count("/") == 1:
                    fixer = c.split("/")
                    if len(fixer[0]) != 0:
                        if fixer[0][-1].isnumeric() and fixer[1][0].isnumeric():
                            pass
                        else:
                            C.insert(C.index(c),fixer[0])
                            if C.index(c) != len(C)-1:
                                C.insert(C.index(c),fixer[1])
                            else:
                                C.append(fixer[1])
                            C[C.index(c)] = "/"

                else:
                    fixer = list(c)
                    for i in range(len(fixer)):
                        if fixer[i] == "/":
                            if fixer[i-1].isnumeric() and fixer[i+1].isnumeric():
                                pass
                            else:
                                C.insert(C.index(c),c[:i])
                                if C.index(c) != len(C)-1:
                                    C.insert(C.index(c)+1,c[i+1:])
                                else:
                                    C.append(c[i+1:])
                                C[C.index(c)] = "/"

            for i in range(len(C)):
                c = C[i].replace(")","").replace("(","")
                C[i] = c

            for i in range(len(C)):
                if C[i].isalpha():
                    C[i] = "{}^1".format(C[i])

            cont = 0
            for i in range(len(C)):
                if C[i] == "/":
                    cont += 1

                else:
                    strmaker = ""
                    z = 0
                    w = 0
                    for j in range(len(C[i])):
                        if C[i][j] == "-" and cont%2 != 0:
                            w = 1
                            pass
                        elif C[i][j].isnumeric() and z == 0:
                            strmaker += "{}".format((-1)**(cont+w)*int(C[i][j]))
                            z = 1
                        else:
                            strmaker += C[i][j]
                    C[i] = strmaker        

            for i in range(C.count("/")):
                C.remove("/")

            for c in C:
                unit = c[:c.find("^")]
                cont = 0
                indexList = []
                for i in range(len(C)):
                    if unit == C[i][:C[i].find("^")]:
                        indexList.append(i)
                        cont += 1
                if cont > 1:
                    disposableObject = MeasureUnit(1,"{}^0".format(unit))
                    for i in indexList:
                        disposableObject *= MeasureUnit(1,C[i])
                    D.append(disposableObject.unit)
                else:
                    D.append(c)
    
            self.unit = D
    
    # Métodos privados. Foram usados nos métodos mágicos
    
    def __powerFixer(self, obj, xI):

        z = 1
        if "-" in obj[xI+1:]:
            xI += 1
            z = -1
        try:
            power = z*float(obj[xI+1:])
        except:                               
            if "/" in obj[xI+1:]:
                top = float(obj[xI+1:obj.find("/")])
                bottom = float(obj[obj.find("/")+1:])
                power = z*top/bottom

        return power
    
    def __finalFormat(self, obj, power):
        if int(power) == power:
            power = int(power)
                            
        xI = obj.find("^") #Corrige o indice em caso "-" tenha aparecido na string
                        
        if power == 0:
            unitFormat = "1"
        else:
            unitFormat = "{}^{}".format(obj[:xI],power)
                        
        return unitFormat
    
    ######################
    #Métodos Públicos:
    
    def isSI(self):
        SI = ["m","s","kg","K"]
        if self.unit in SI:
            return True
        
    def ans(self):
        if len(self.unit) == 1:
            self.unit = self.unit[0]
            
        return (self.value,self.unit)
    
    ######################
    #Métodos Mágicos:
    
    def __eq__(self,other):
        if len(self.unit) == len(other.unit):
            cont = 0
            for i in self.unit:
                if i in other.unit:
                    cont += 1
            if cont == len(self.unit):
                return True
            else:
                return False
    
    def __add__(self,other):
        if self == other:
            return MeasureUnit(self.value + other.value,self.unit)
        else:
            print("Essa Operação não tem sentido físico")
    
    def __sub__(self,other):
        if self == other:
            return MeasureUnit(self.value - other.value,self.unit)
        else:
            print("Essa Operação não tem sentido físico")
    
    def __pow__(self,power):
        powUnits = []
        for sf in self.unit:
            sfI = sf.find("^")
            powUnitSelf = self.__powerFixer(sf,sfI)
            unitFormat = self.__finalFormat(sf,powUnitSelf*power)
            
            powUnits.append(unitFormat)
        
        if "1" in powUnits and len(powUnits) > 1:
            for i in range(powUnits.count("1")-1):
                powUnits.remove("1")
        
        for i in range(len(powUnits)):
            if powUnits[i][0] == "^":
                powUnits[i] = "1"
                
        return MeasureUnit(self.value**power,powUnits)
    
    def __mul__(self,other):
        
        finalunit = []
        
        for sf in self.unit:
            for ot in other.unit:
                sfI = sf.find("^") #Index of "^" in string
                otI = ot.find("^")
                
                if sf[:sfI] == ot[:otI]:
                    if sf[:sfI] == "1":
                        finalunit.append("1")
                    else:
                        powUnitSelf = self.__powerFixer(sf,sfI)
                        powUnitOther = other.__powerFixer(ot,otI)
                    
                        totalpow = powUnitSelf + powUnitOther
                        unitFormat = self.__finalFormat(sf,totalpow)
                        
                        finalunit.append(unitFormat)
            
        if len(finalunit) == 0:
            finalunit = self.unit + other.unit
        
        if "1" in finalunit and len(finalunit) > 1:
            for i in range(finalunit.count("1")-1):
                finalunit.remove("1")
        
        return MeasureUnit(self.value*other.value,finalunit)
    
    def __truediv__(self,other):
                
        finalunit = []
        
        for sf in self.unit:
            for ot in other.unit:
                sfI = sf.find("^") #Index of "^" in string
                otI = ot.find("^")
                
                if sf[:sfI] == ot[:otI]:
                    if sf[:sfI] == "1":
                        finalunit.append("1")
                    else:
                        powUnitSelf = self.__powerFixer(sf,sfI)
                        powUnitOther = other.__powerFixer(ot,otI)
                        
                        totalpow = powUnitSelf - powUnitOther
                        unitFormat = self.__finalFormat(sf,totalpow)
                        
                        finalunit.append(unitFormat)
        
        if len(finalunit) == 0:
            disposableObject = MeasureUnit(1,other.unit)
            disposableObject = disposableObject**-1    
            finalunit = self.unit + disposableObject.unit
        
        if "1" in finalunit and len(finalunit) > 1:
            for i in range(finalunit.count("1")-1):
                finalunit.remove("1")
        
        return MeasureUnit(self.value/other.value,finalunit)

In [187]:
a = MeasureUnit(600,'1')
b = MeasureUnit(300,"m")
c = MeasureUnit(500,"m^2")
d = MeasureUnit(200, "J*kg/K")
e = MeasureUnit(400, "m**3/s")

In [188]:
a = MeasureUnit(200, "m")
b = MeasureUnit(300, "m")
print(a == b)
print((a*b).ans())
print((a/b).ans())
print((a**-1).ans())
print((b**2).ans())

True
(60000, 'm^2')
(0.6666666666666666, '1')
(0.005, 'm^-1')
(90000, 'm^2')


In [161]:
"1" in ['1', 'm^1']

True